# Libraries Loading

In [23]:
import plotly.express as px
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import torchvision.transforms as T
import os
from pathlib import Path
import sklearn
import librosa
import librosa.display
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision.utils import make_grid
from utils import pytorchtools
from utils import preprocessing
import torch
import seaborn as sns
from tqdm import tqdm
from utils.preprocessing import plot_mel
%matplotlib inline

# Feature Engineering & Modeling

In [24]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

sample_rate, samples = wavfile.read('../data/raw/wav/09b03Ed.wav')
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

In [25]:
# make audio player to play the audio
from IPython.display import Audio
Audio(samples, rate=sample_rate)

In [31]:
image_dataset_path = Path(Path().resolve().parents[0], "data/preprocessed/images/")
dataset_exists = os.path.exists(image_dataset_path)
if not dataset_exists:
  # Create a new directory because it does not exist 
  os.makedirs(image_dataset_path)
  print("directory is created!")
  print(image_dataset_path)

In [ ]:
images_count = len(os.listdir(image_dataset_path))
audio_count =  len(os.listdir(audio_dataset_path))

NameError: name 'audio_dataset_path' is not defined

In [ ]:
if images_count != audio_count:
    for path in tqdm(dataset_summary['path']):
        jpeg_path = path.replace('raw','preprocessed/images').replace('jpeg/',"").replace('wav','png')
        audio, rate = librosa.load(path)
        fig = plot_mel(audio,rate)
        fig.savefig(jpeg_path)
#         # convert it to a numpy array
#         numpy_image = mplfig_to_npimage(fig)
#         torch.tensor(numpy_image)

  0%|          | 0/535 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../data/preprocessed/images/png/16a02Lb.png'

In [ ]:
dataset_summary = decompose_emodb(image_dataset_path)

In [ ]:
labels = dataset_summary.labels.unique()
le = sklearn.preprocessing.LabelEncoder()
targets = le.fit_transform(labels)
le.inverse_transform(targets)

array([], dtype=float64)

In [ ]:
preprocessing.label_encoder_to_json(le,'labels.json')

In [ ]:
dataset_summary.replace(labels,le.fit_transform(labels),inplace=True)

In [ ]:
dataset_summary

,labels,path


In [ ]:
targets

In [ ]:
class EmoDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, idx):
        
        image_label = self.df.iloc[idx]['labels']
        image_path = self.df.iloc[idx]['path']
        
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)
        if image.shape[0] == 1:
            image = image.repeat(3, 1, 1)

        return (image, image_label)

In [ ]:
EmoDataset(dataset_summary)

AttributeError: 'EmoDataset' object has no attribute 'labels'

In [ ]:
from sklearn.model_selection import train_test_split

# 0.8 * 0.125 gives 0.1 for validation
train_df,test_df = train_test_split(dataset_summary, test_size=0.2,stratify=dataset_summary.labels)
train_df,valid_df = train_test_split(train_df,test_size=0.125,stratify=train_df.labels)

In [ ]:
sns.countplot(x ='labels',data=train_df)
#replace with plotly countplot 

In [ ]:
sns.countplot(x ='labels',data=test_df)

In [ ]:
sns.countplot(x ='labels',data=valid_df)

In [ ]:
train_df.reset_index(inplace=True,drop=True)
test_df.reset_index(inplace=True,drop=True)
valid_df.reset_index(inplace=True,drop=True)

In [ ]:
resize = T.Compose([T.Resize((256,256)),
                             T.ToTensor()])

In [ ]:
train_dataset = EmoDataset(train_df, transform=resize)
test_dataset = EmoDataset(test_df, transform=resize)
valid_dataset = EmoDataset(valid_df, transform=resize)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32,shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=32,shuffle=True)

In [ ]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:32], nrow=8).permute(1, 2, 0))
        break

In [ ]:
show_batch(valid_loader)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
import torch as torch
from torchvision import transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## USE APPLE M1 GPU
try:
    if torch.backends.mps.is_available():
        device = torch.device('mps')
except AttributeError:
    print(device)
        

In [ ]:
for d,l in valid_loader:
    print(len(l))

In [ ]:
import torch
# model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)


##REPLACE LAST LAYER
num_labels = len(dataset_summary['labels'].unique())
model.classifier[6] = torch.nn.Linear(4096,num_labels)
#Freeze the gradients of all of the layers in the features (convolutional) layers
# for param in model.features.parameters():
#     param.requires_grad = False



params = {'learning_rate':1e-5,'epochs':500, 'patience':20}
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
running_loss = 0
losses = []

In [ ]:
model.to(device)

In [ ]:
# to track the training loss as the model trains
train_losses = []
# to track the validation loss as the model trains
valid_losses = []
# to track the average training loss per epoch as the model trains
avg_train_losses = []
# to track the average validation loss per epoch as the model trains
avg_valid_losses = []

In [ ]:
def testAccuracy(model, test_loader, device):
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for images,labels in test_loader:
            # run the model on the test set to predict labels
            images=images.to(device)
            labels = labels.to(device)
            outputs = model(images).to(device)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (accuracy / total)
    return accuracy

In [ ]:
size = len(train_loader.dataset)
print(size)

# initialize the early_stopping object
early_stopping = pytorchtools.EarlyStopping(patience=params['patience'],verbose=True)

for epoch in range(params['epochs']):
    #######TRAIN MODEL########
    epochs_loss=0

    model.train()
    for i, (images,labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images).to(device)
        loss = criterion(outputs, labels)

        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        #calculate train_loss
        train_losses.append(loss.item())
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0, norm_type=2)
        optimizer.step()
    ##########################    
    #####TEST MODEL#######
    ##########################    
    accuracy = testAccuracy(model, test_loader, device)
    ##########################    
    #####VALIDATE MODEL#######
    ##########################
    model.eval()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images).to(device)
        loss = criterion(outputs,labels)
        valid_losses.append(loss.item())
    
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    #print(train_loss)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)

    print_msg = (f'accuracy: {accuracy:.3f} ' + f'train_loss: {train_loss:.3f} ' + f'valid_loss: {valid_loss:.3f}')
    
    print(print_msg)

    # clear lists to track next epoch
    train_losses = []
    valid_losses = []
    
    early_stopping(valid_loss, model)
    print(epoch)
        
    if early_stopping.early_stop:
        print("Early stopping")
        break

In [ ]:
def error_analysis(test_loader, model, label_encoder, device):
    model.eval()
    preds = []
    true_labels = []
    for images, labels in test_loader:

        data, target = images.to(device), labels.to(device)
        output = model(data)  # shape = torch.Size([batch_size, 10])
        pred = output.argmax(
            dim=1, keepdim=True
        )  # pred will be a 2d tensor of shape [batch_size,1]

        preds.append(pred.flatten().to("cpu").numpy())
        true_labels.append(labels.flatten().to("cpu").numpy())

    #### GET MISIDENTIFIED EXAMPLE
    all_preds = np.concatenate(preds, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    df = pd.DataFrame({"expected": true_labels, "predicted": all_preds})

    ### DECODE LABELS in datafram
    for col in df:
        df[col] = label_encoder.inverse_transform(df[col])

    return df

In [ ]:
from collections import defaultdict

errs = error_analysis(test_loader,model, le, device)

In [ ]:
errs

# Results & Visualizations

In [ ]:
fig = plt.figure(figsize=(20,10))
######1st plot#########
ax1 = fig.add_subplot()
ax1.set_ylabel('valid  /  train loss')
ax1.set_xlabel('number of epochs')
halt = avg_valid_losses.index(min(avg_valid_losses))


plt.axvline(x=halt, color='r', linestyle="--", label="stop training")

print(avg_valid_losses.index(min(avg_valid_losses)))


plt.plot(list(range(len(avg_valid_losses))), avg_valid_losses, label = "valid loss")
plt.plot(list(range(len(avg_valid_losses))), avg_train_losses, label="train loss")


h,labels = ax1.get_legend_handles_labels()
labels[:1] = ['stop training','valid loss', 'train loss',]
ax1.legend(labels=labels)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns

def generate_confusion_matrix(y_true,y,classes):
    cf_matrix = confusion_matrix(y_true, y)
    plt.figure(figsize=(15, 10))
    ax= plt.subplot()
    sns.heatmap(cf_matrix, annot=True)
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(classes)
    ax.yaxis.set_ticklabels(classes)
    plt.savefig('./matrix.jpeg')

In [ ]:
generate_confusion_matrix(errs['expected'], errs['predicted'],le.classes_)